In [82]:
import torch
import sys
import gc
sys.path.append('..')
%load_ext autoreload
%autoreload 2
from train_trl import TrainerWrapper, RLConfig, MOCK_LLAMA, LLAMA_3_2_1B

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [83]:
cfg = RLConfig(single_process_mode=True, model_id=MOCK_LLAMA)
wrapper = TrainerWrapper(cfg)
wrapper.init_model()
wrapper.init_data_module()
wrapper.init_trainer()

2024-11-24 19:56:40.920 | INFO     | train_trl:init_model:48 - Loading model qgallouedec/tiny-LlamaForCausalLM-3


2024-11-24 19:57:04.915 | INFO     | dataset.squad:setup:220 - Loading dataset for stage fit
2024-11-24 19:57:06.134 | INFO     | dataset.squad:setup:224 - Loaded dataset with 60917 samples
2024-11-24 19:57:06.148 | INFO     | dataset.squad:setup:232 - Processing dataset for stage fit, workers: 1, cache dir dataset_caches/ultrafeedback
Map: 100%|██████████| 100/100 [00:00<00:00, 2269.49 examples/s]
/admin/home-brianf/smolmodels/.venv/lib/python3.11/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Tokenizing eval dataset: 100%|██████████| 100/100 [00:00<00:00, 326.62 examples/s]


In [84]:
gc.collect()
torch.cuda.empty_cache()

In [87]:
wrapper.compute_loss_metrics()

Computing DPO loss:   0%|          | 0/900 [00:00<?, ?it/s]

/admin/home-brianf/smolmodels/.venv/lib/python3.11/site-packages/trl/trainer/dpo_trainer.py:122: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  prompt_input_ids = [torch.tensor(example["prompt_input_ids"]) for example in examples]
/admin/home-brianf/smolmodels/.venv/lib/python3.11/site-packages/trl/trainer/dpo_trainer.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  chosen_input_ids = [torch.tensor(example["chosen_input_ids"]) for example in examples]
/admin/home-brianf/smolmodels/.venv/lib/python3.11/site-packages/trl/trainer/dpo_trainer.py:126: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requi

KeyboardInterrupt: 

In [ ]:
# plot distributiuon of losses
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

STR_COLS = ['prompt', 'chosen', 'rejected']

out_df = pd.DataFrame(outputs)

for col in out_df.columns:
    if col in STR_COLS:
        continue
    print(out_df[col].describe())
    plt.hist(out_df[col], bins=1000)
    plt.title(col)
    plt.show()